## Step 1: Restructure directory to match pytorch image dataset structure

In [1]:
import os

def restructure(dir, num_classes):
    for i in range(1, num_classes+1):
        dir_to_create = f".\\{dir}\\{str(i)}"
        
        if not os.path.isdir(dir_to_create):
            os.mkdir(dir_to_create)

    cwd = os.path.abspath(dir)

    for file in os.listdir(dir):
        if file.endswith(".jpg"):
            os.replace(f"{cwd}\\{file}", f"{cwd}\\{file.split("_")[0]}\\{file}")

restructure("train_data", 60)
restructure("val_data", 60)

## Step 2: Load dataset into torch dataloader

In [3]:
import torch.utils.data
from torchvision import datasets, transforms

train_dataset = datasets.ImageFolder("train_data", transform=transforms.ToTensor())
val_dataset = datasets.ImageFolder("val_data", transform=transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=True)